In [1]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/raw/main/04-monitoring/data/results-gpt4o-mini.csv'

In [2]:
import pandas as pd
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [4]:
df = df.iloc[:300]

In [ ]:
df

In [21]:
from sentence_transformers import SentenceTransformer,util

In [10]:
model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

/home/oleg/llm-workshop/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
answer_llm = df.iloc[0].answer_llm

In [18]:
answer_orig = df.iloc[0].answer_orig

In [14]:
vector = model.encode(answer_llm)

In [16]:
vector[0]

np.float32(-0.42244673)

### Q1
answer: np.float32(-0.42244673)

In [81]:
llm_and_orig_answers = df.to_dict(orient='records')

In [82]:
row = llm_and_orig_answers[0]

In [83]:
def compute_dot_similarity(row):
    answer_llm_emb = model.encode(row['answer_llm'])
    answer_orig_emb = model.encode(row['answer_orig'])

    return util.dot_score(answer_llm_emb, answer_orig_emb).cpu().tolist()


In [86]:
from tqdm import tqdm

In [91]:
evaluations = []

for row in tqdm(llm_and_orig_answers):
    dot_similarity = compute_dot_similarity(row)
    evaluations.append(dot_similarity)


100%|██████████| 300/300 [00:05<00:00, 53.97it/s]


In [ ]:
evaluations

In [93]:
import numpy as np

In [94]:
# Calculate the 75th percentile
percentile_75 = np.percentile(evaluations, 75)

In [95]:
percentile_75

np.float64(31.6743106842041)

### Q2
np.float64(31.6743106842041)

In [97]:
def compute_cosine_similarity(row):
    answer_llm_emb = model.encode(row['answer_llm'])
    norm = np.sqrt((answer_llm_emb * answer_llm_emb).sum())
    norm_llm = answer_llm_emb / norm

    answer_orig_emb = model.encode(row['answer_orig'])
    norm = np.sqrt((answer_orig_emb * answer_orig_emb).sum())
    norm_orig = answer_orig_emb / norm


    return util.dot_score(norm_llm, norm_orig).cpu().tolist()


In [98]:
evaluations_normalized = []

for row in tqdm(llm_and_orig_answers):
    cosine_similarity = compute_cosine_similarity(row)
    evaluations_normalized.append(cosine_similarity)


100%|██████████| 300/300 [00:05<00:00, 53.04it/s]


In [99]:
cosine_percentile_75 = np.percentile(evaluations_normalized, 75)

In [100]:
cosine_percentile_75

np.float64(0.8362348079681396)

### Q3
np.float64(0.8362348079681396)

In [103]:
from rouge import Rouge
rouge_scorer = Rouge()

In [110]:
rouge_scores = rouge_scorer.get_scores(llm_and_orig_answers[10]['answer_llm'], llm_and_orig_answers[10]['answer_orig'])[0]

In [111]:
rouge_scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

### Q4
  'f': 0.45454544954545456},

In [112]:
f_scores = [rouge_scores['rouge-1']['f'], rouge_scores['rouge-2']['f'], rouge_scores['rouge-l']['f']]
average_f_score = sum(f_scores) / len(f_scores)
average_f_score

0.35490034990035496

### Q5
0.35490034990035496

In [114]:
def compute_rouge_scores(row):
    return rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])

In [115]:
all_rouge_scores = []

for row in llm_and_orig_answers:
    row_rouge_score = compute_rouge_scores(row)

    all_rouge_scores.append(row_rouge_score)

In [ ]:
all_rouge_scores

What's the average F-score in rouge_2 across all the records?

In [121]:
f_scores_rouge_2 = [record[0]['rouge-2']['f'] for record in all_rouge_scores]
average_f_score_rouge_2 = sum(f_scores_rouge_2) / len(f_scores_rouge_2)

average_f_score_rouge_2

0.20696501983423318

### Q6
0.20696501983423318